# Decorators

### What are Decorators?
In Python, a decorator is a function that accepts a function and returns a function.
Basically, a decorator allows us to modify the behavior of a function without modifying the function's actual code.

Functions are first class objects (functions can be used or passed as arguments).
Properties of first class functions:
* store the function in a variable
* pass the function as a parameter to another function
* return the function from a function
* store them in data structures such as lists

In [8]:
# returning call to wrapper function
def main_func(something):
    def wrapper():
        print('Started wrapper function')
        print(something)
        print('Ended wrapper function')
    return wrapper()

main_func('Something')

Started wrapper function
Something
Ended wrapper function


In [9]:
# returning a wrapper function object
def main_func(something):
    def wrapper():
        print('Started wrapper function')
        print(something)
        print('Ended wrapper function')
    return wrapper

var_x = main_func('Something')
print(var_x)
var_x()

<function main_func.<locals>.wrapper at 0x11056bb00>
Started wrapper function
Something
Ended wrapper function


In [10]:
# pass a function as parameter to main_func
def main_func(f):
    def wrapper():
        print('Started wrapper function')
        f()
        print('Ended wrapper function')
    return wrapper

def display_func():
    print('This is display_func')

display_func = main_func(display_func)
print(display_func)
display_func()

<function main_func.<locals>.wrapper at 0x11056bf80>
Started wrapper function
This is display_func
Ended wrapper function


### Example of a Decorator

In [11]:
# decorator function
def main_func(f):
    def wrapper():
        print('Started wrapper function')
        f()
        print('Ended wrapper function')
    return wrapper

@main_func
def display_func():
    print('This is display_func')

print(display_func)
display_func()

<function main_func.<locals>.wrapper at 0x1105803b0>
Started wrapper function
This is display_func
Ended wrapper function


##### What about positional arguments?

In [12]:
# decorator function
def main_func(f):
    def wrapper():
        print('Started wrapper function')
        f()
        print('Ended wrapper function')
    return wrapper

@main_func
def display_func(x):
    print(x)

display_func('This is display_func')

TypeError: wrapper() takes 0 positional arguments but 1 was given

In [13]:
# Let's solve the above error for good!
def main_func(f):
    def wrapper(*arg, **kwarg):
        print('Started wrapper function')
        f(*arg, **kwarg)
        print('Ended wrapper function')
    return wrapper

@main_func
def display_func(x, y):
    print(x, y)

display_func('This is display_func.', 'Done!')

Started wrapper function
This is display_func. Done!
Ended wrapper function


##### What about return values?

In [15]:
def main_func(f):
    def wrapper(*arg, **kwarg):
        print('Started wrapper function')
        f(*arg, **kwarg)
        print('Ended wrapper function')
    return wrapper

@main_func
def display_func(x, y):
    print(x)
    return y

display_func('This is display_func', 'Done')

Started wrapper function
This is display_func
Ended wrapper function


In [16]:
# Fixed it by string return value in rv and return rv variable in wrapper function
def main_func(f):
    def wrapper(*args, **kwargs):
        print('Started wrapper function')
        rv = f(*args, **kwargs)
        print('Ended wrapper function')
        return rv
    return wrapper

@main_func
def display_func(x, y):
    print(x)
    return y

x = display_func('This is display_func', 'Done')
print(x)

Started wrapper function
This is display_func
Ended wrapper function
Done


### Create a Custom Decorator
A decorator that times the runtime of a function.

In [18]:
import logging
import os
from time import time

def timer(func):
    cwd = os.getcwd()
    log_file = os.path.join(cwd, 'timer.log')
    for handler in logging.root.handlers[:]:
        logging.root.removeHandler(handler)
    logging.basicConfig(filename=log_file, filemode='a', format='%(asctime)s %(levelname)s %(message)s', datefmt='%H:%M:%S', level=logging.INFO)

    def wrapper(*args, **kwargs):
        try:
            start = time()
            rv = func(*args, **kwargs)
            end = time() - start
            logging.info(f' {func.__name__} ran with args: {args} and kwargs: {kwargs} and took {end} sec to run.')
            return rv
        except Exception as e:
            logging.error(f' {func.__name__} ran into an error: {e}.')
    return wrapper

@timer
def test_range(start, end):
    # 100 million records
    for _ in range(start, end):
        pass

@timer
def test_range1(start, end):
    # 10 million records
    for _ in range(start, end):
        pass

test_range(0, 100000000)
test_range1(0, 10000000)